In [1]:
import csv
import pandas
import numpy
import random
from CWHashingFamily import CWHashingFamily

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [6]:
ID = 0
TEXT = 9
R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv",minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.reader(x))

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.map(lambda line: (int(line[ID]), shingles(line[TEXT].encode('utf-8'),SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] ))



NameError: name 'sc' is not defined

In [5]:
from LSH import LSH
from Record import Record

lsh = LSH(R,B)
#ESTO NO ANDA

trainList = trainRDD.take(100)

testRDD = sc.parallelize(trainList)

testRDD = testRDD.foreach(lambda line: lsh.add( Record(line[0], line[1]) ) )

#ESTO ANDA:
#trainRDD = trainRDD.take(10)
#for i in xrange(len(trainRDD)):
#    record = Record((trainRDD[i][0], trainRDD[i][1]) )
#    lsh.add(record)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 98, in main
    command = pickleSer._read_with_length(infile)
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 164, in _read_with_length
    return self.loads(obj)
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 422, in loads
    return pickle.loads(obj)
UnpicklingError: NEWOBJ class argument has NULL tp_new

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1

In [17]:
shinglesText = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."
test = (492346,shingles(shinglesText,SHINGLES_LEN))
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ])
testRecord = Record(test)

In [18]:
print lsh.getAllSimilarRecords(testRecord)

print lsh.getTable(0)